In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("data.csv")
#df

# Seller Column

# Column Name = Make

In [ ]:
df["make"]

In [4]:
df["make"]=df["make"].str.strip(',\n')
df["make"]

# Column Name  = model

In [6]:
df["model"]

0         \n,A3,\n
1         \n,A3,\n
2         \n,A1,\n
3         \n,A1,\n
4         \n,A3,\n
           ...    
62761    \n,S60,\n
62762    \n,S60,\n
62763    \n,S60,\n
62764    \n,S40,\n
62765    \n,S60,\n
Name: model, Length: 62766, dtype: object

In [ ]:
df["model"]=df["model"].str.strip(',\n')
df["model"]